In [9]:
from sqlalchemy import create_engine, text

In [11]:
engine = create_engine('sqlite:///ecommerce.db', echo=True)

In [ ]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT 1"))
    # Fetch and print the result
    print(result.scalar()) # This should print 1 if the connection is successful

In [13]:
from sqlalchemy import (
    create_engine,
    Table,
    Column,
    Integer,
    String,
    Float,
    ForeignKey,
    DateTime,
    text,
)
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime

Base = declarative_base()


class Book(Base):
    __tablename__ = "books"

    book_id = Column(Integer, primary_key=True)
    title = Column(String(200), nullable=False)
    author = Column(String(100), nullable=False)
    isbn = Column(String(13), unique=True)
    price = Column(Float, nullable=False)
    stock = Column(Integer, default=0)
    category = Column(String(50))


class Customer(Base):
    __tablename__ = "customers"

    customer_id = Column(Integer, primary_key=True)
    email = Column(String(100), unique=True, nullable=False)
    name = Column(String(100), nullable=False)
    address = Column(String(200))
    phone = Column(String(20))


class Order(Base):
    __tablename__ = "orders"

    order_id = Column(Integer, primary_key=True)
    customer_id = Column(Integer, ForeignKey("customers.customer_id"))
    order_date = Column(DateTime, default=datetime.utcnow)
    total_amount = Column(Float, nullable=False)
    status = Column(String(20), default="pending")


class OrderItem(Base):
    __tablename__ = "order_items"

    item_id = Column(Integer, primary_key=True)
    order_id = Column(Integer, ForeignKey("orders.order_id"))
    book_id = Column(Integer, ForeignKey("books.book_id"))
    quantity = Column(Integer, nullable=False)
    price = Column(Float, nullable=False)


# Create all tables in the database
engine = create_engine("sqlite:///bookstore.db", echo=True)
Base.metadata.create_all(engine)

2024-11-11 14:17:29,880 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-11 14:17:29,882 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("books")
2024-11-11 14:17:29,886 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-11 14:17:29,893 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("books")
2024-11-11 14:17:29,900 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-11 14:17:29,913 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customers")
2024-11-11 14:17:29,914 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-11 14:17:29,917 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("customers")
2024-11-11 14:17:29,918 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-11 14:17:29,922 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2024-11-11 14:17:29,923 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-11 14:17:29,928 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("orders")
2024-11-11 14:17:29,930 INFO sqlalchemy.engine.Engine [raw sql] ()
2

/var/folders/0n/m62rpbbj60b2p0xcy2j637800000gn/T/ipykernel_43827/2704103804.py:15: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [14]:
with engine.connect() as connection:
    result = connection.execute(
        text("SELECT name FROM sqlite_master WHERE type='table';")
    )
    print("\nTables created:")
    for row in result:
        print(row[0])

2024-11-11 14:18:03,164 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-11 14:18:03,197 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table';
2024-11-11 14:18:03,206 INFO sqlalchemy.engine.Engine [generated in 0.04242s] ()

Tables created:
books
customers
orders
order_items
2024-11-11 14:18:03,219 INFO sqlalchemy.engine.Engine ROLLBACK


In [15]:
from sqlalchemy.orm import Session


# Create sample data
def populate_sample_data():
    with Session(engine) as session:
        # Add books
        books = [
            Book(
                title="The Great Gatsby",
                author="F. Scott Fitzgerald",
                isbn="9780743273565",
                price=14.99,
                stock=50,
                category="Fiction",
            ),
            Book(
                title="Python Crash Course",
                author="Eric Matthes",
                isbn="9781593279288",
                price=29.99,
                stock=35,
                category="Programming",
            ),
            Book(
                title="Dune",
                author="Frank Herbert",
                isbn="9780441172719",
                price=18.99,
                stock=25,
                category="Science Fiction",
            ),
        ]
        session.add_all(books)

        # Add customers
        customers = [
            Customer(
                email="john@example.com",
                name="John Smith",
                address="123 Main St",
                phone="555-0123",
            ),
            Customer(
                email="jane@example.com",
                name="Jane Doe",
                address="456 Oak Ave",
                phone="555-0456",
            ),
        ]
        session.add_all(customers)

        # Commit to save books and customers first
        session.commit()

        # Add orders
        order1 = Order(customer_id=1, total_amount=44.98, status="completed")
        session.add(order1)
        session.flush()  # This ensures order1 has an ID

        # Add order items
        order_items = [
            OrderItem(order_id=order1.order_id, book_id=1, quantity=2, price=14.99),
            OrderItem(order_id=order1.order_id, book_id=2, quantity=1, price=29.99),
        ]
        session.add_all(order_items)

        session.commit()


# Run the population function
try:
    populate_sample_data()
    print("Sample data inserted successfully!")
except Exception as e:
    print(f"Error inserting data: {e}")

# Test query to verify the data
with Session(engine) as session:
    # Query to show books ordered by a customer
    query = text(
        """
        SELECT c.name, b.title, oi.quantity, oi.price
        FROM customers c
        JOIN orders o ON c.customer_id = o.customer_id
        JOIN order_items oi ON o.order_id = oi.order_id
        JOIN books b ON oi.book_id = b.book_id
        WHERE c.customer_id = 1;
    """
    )

    result = session.execute(query)
    print("\nOrders for John Smith:")
    print("Customer | Book | Quantity | Price")
    print("-" * 50)
    for row in result:
        print(f"{row.name} | {row.title} | {row.quantity} | ${row.price}")

2024-11-11 14:19:20,073 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-11 14:19:20,088 INFO sqlalchemy.engine.Engine INSERT INTO books (title, author, isbn, price, stock, category) VALUES (?, ?, ?, ?, ?, ?) RETURNING book_id
2024-11-11 14:19:20,093 INFO sqlalchemy.engine.Engine [generated in 0.00235s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('The Great Gatsby', 'F. Scott Fitzgerald', '9780743273565', 14.99, 50, 'Fiction')
2024-11-11 14:19:20,103 INFO sqlalchemy.engine.Engine INSERT INTO books (title, author, isbn, price, stock, category) VALUES (?, ?, ?, ?, ?, ?) RETURNING book_id
2024-11-11 14:19:20,104 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('Python Crash Course', 'Eric Matthes', '9781593279288', 29.99, 35, 'Programming')
2024-11-11 14:19:20,108 INFO sqlalchemy.engine.Engine INSERT INTO books (title, author, isbn, price, stock, category) VALUES (?, ?, ?, ?, ?, ?) RETURNING book_id
2024-11-11 14:19:20,113 INFO sqlalche